# Data source

The data for this notebook is pulled from https://www.alltime-athletics.com/m_100ok.htm

Unfortunately, they do not provide the data in any downloadable format, so I have just copied their table to `mens-100m.txt`.

# Converting to CSV for future convenience

In [1]:
import os
import re

def create_csv(input_path, output_path):
    processed_lines = []
    with open(input_path, 'r') as data:
        for line in data:
            line = line.strip()
            columns = re.split(r'\s{2,}', line)
            processed_lines.append('"' + '","'.join(columns) + '"')
    
    with open(output_path, 'w') as file:
        file.write('\n'.join(processed_lines))

ALWAYS_REMAKE = False
input_path = 'mens-100m.txt'
output_path = 'mens-100m.csv'
if ALWAYS_REMAKE or not os.path.isfile(output_path):
    print('Creating CSV File...')
    create_csv(input_path, output_path)

# Loading the data

In [2]:
import pandas as pd

dt = pd.read_csv('mens-100m.csv', header=None, quotechar='"')
dt.head()

,0,1,2,3,4,5,6,7,8
0,1,9.58,+0.9,Usain Bolt,JAM,21.08.86,1,Berlin,16.08.2009
1,2,9.63,+1.5,Usain Bolt,JAM,21.08.86,1,London,05.08.2012
2,3,9.69,±0.0,Usain Bolt,JAM,21.08.86,1,Beijing,16.08.2008
3,3,9.69,+2.0,Tyson Gay,USA,09.08.82,1,Shanghai,20.09.2009
4,3,9.69,-0.1,Yohan Blake,JAM,26.12.89,1,Lausanne,23.08.2012
